# Thesis Extraction for Manual Review

This notebook:
1. Loads PDF files from `data/latest_thesis_decks`
2. Uses GPT-4 to extract all thesis statements
3. Numbers each thesis uniquely
4. Generates 2-3 sentence descriptions
5. Identifies which theses support other theses
6. Exports to CSV for easy human review

**Output:** A clean CSV file with all theses ready for manual review and refinement.


In [13]:
import os
import json
import re  # For JSON extraction
import time  # For waiting on assistant runs
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any
import fitz  # PyMuPDF
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
env_path = Path("../.env").resolve()
load_dotenv(env_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

DATA_FOLDER = Path("../data/latest_thesis_decks")
OUTPUT_FOLDER = Path("../data/analysis")
OUTPUT_FOLDER.mkdir(exist_ok=True, parents=True)

print("✓ Imports and configuration loaded")
print(f"  Data folder: {DATA_FOLDER.absolute()}")
print(f"  Output folder: {OUTPUT_FOLDER.absolute()}")
print(f"  OpenAI client: {'✓' if openai_client else '✗'}")


✓ Imports and configuration loaded
  Data folder: C:\Users\ChetKumar\Projects\thesis-bot\notebooks\..\data\latest_thesis_decks
  Output folder: C:\Users\ChetKumar\Projects\thesis-bot\notebooks\..\data\analysis
  OpenAI client: ✓


In [ ]:
# Step 1: Load PDFs and extract text
# We extract text ourselves to ensure the full document is analyzed

def extract_text_from_pdf(pdf_path: Path) -> str:
    """Extract all text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc[page_num]
        text += page.get_text()
    doc.close()
    return text

def load_all_pdfs(data_folder: Path) -> Dict[str, str]:
    """Load all PDF files from the data folder and extract their text."""
    pdf_texts = {}
    pdf_files = list(data_folder.glob("*.pdf"))
    
    if not pdf_files:
        print(f"⚠ No PDF files found in {data_folder}")
        return pdf_texts
    
    for pdf_file in pdf_files:
        print(f"Extracting text from: {pdf_file.name}")
        try:
            text = extract_text_from_pdf(pdf_file)
            pdf_texts[pdf_file.name] = text
            print(f"  ✓ Extracted {len(text):,} characters")
        except Exception as e:
            print(f"  ✗ Error extracting {pdf_file.name}: {e}")
    
    return pdf_texts

# Load all PDFs
pdf_texts = load_all_pdfs(DATA_FOLDER)
print(f"\n✓ Loaded {len(pdf_texts)} PDF file(s)")


✓ Found 4 PDF file(s) ready for upload
  - AI 20250923.pdf
  - Argonautic BioTech Presentation 20250412.pdf
  - Argonautic Consolidated Thesis Presentation 20250818.pdf
  - Argonautic Construction Technology 20250412.pdf


In [14]:
# Step 2: Extract theses using GPT-4 Turbo with full document text

def extract_theses_from_text(text: str, filename: str) -> Dict[str, Any]:
    """
    Use GPT-4 Turbo to extract theses from the full document text.
    GPT-4 Turbo has 128K token context, so we can send the entire document.
    """
    if not openai_client:
        print("⚠ OpenAI client not configured.")
        return {"theses": [], "thesis_supports": []}
    
    # GPT-4 Turbo: 128K tokens ≈ 500K characters
    # Reserve ~10K tokens for prompt/system, so ~470K characters for text
    max_chars = 470000
    
    # If document is longer, we'll process in chunks
    if len(text) > max_chars:
        print(f"  ⚠ Document is very long ({len(text):,} chars). Processing in chunks...")
        return extract_theses_from_text_chunked(text, filename, max_chars)
    
    prompt = f"""Analyze the following document and extract ALL thesis statements/arguments.

A thesis is a claim, argument, or proposition that the document is making. Extract every distinct thesis statement.

For each thesis, provide:
1. The exact thesis statement (the claim being made)
2. A brief 2-3 sentence description that accurately describes what the thesis is about

Also identify relationships: if one thesis supports or provides evidence for another thesis, note that.

IMPORTANT:
- Analyze the ENTIRE document from beginning to end
- Extract ALL theses, even if they seem similar (we will deduplicate later)
- Be thorough - don't miss any arguments or claims
- Each thesis should be a distinct statement
- If theses are redundant or very similar, still list them (we'll handle deduplication)

Return a JSON object with this structure:
{{
    "theses": [
        {{
            "thesis": "The exact thesis statement",
            "description": "2-3 sentences accurately describing what this thesis is about"
        }},
        ...
    ],
    "thesis_supports": [
        {{
            "source_thesis": "The supporting thesis statement",
            "target_thesis": "The thesis statement it supports"
        }},
        ...
    ]
}}

Document text (FULL DOCUMENT - analyze everything):
{text}
"""
    
    try:
        print(f"  Analyzing document ({len(text):,} characters)...")
        response = openai_client.chat.completions.create(
            model="gpt-4-turbo-preview",  # or "gpt-4o" - both have 128K context
            messages=[
                {"role": "system", "content": "You are an expert at analyzing documents and extracting thesis statements. Always return valid JSON. Be thorough and extract ALL thesis statements from the entire document, even if they seem similar."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=4096  # Maximum supported by gpt-4-turbo-preview
        )
        
        result_text = response.choices[0].message.content
        
        # Try to extract JSON from the response
        if "```json" in result_text:
            result_text = result_text.split("```json")[1].split("```")[0]
        elif "```" in result_text:
            result_text = result_text.split("```")[1].split("```")[0]
        
        # Find JSON object in the text
        json_match = re.search(r'\{.*\}', result_text, re.DOTALL)
        if json_match:
            result_text = json_match.group(0)
        
        result = json.loads(result_text.strip())
        result["filename"] = filename
        return result
    except json.JSONDecodeError as json_err:
        print(f"  ⚠ Failed to parse JSON: {json_err}")
        print(f"  Debug: Response (first 1000 chars): {result_text[:1000] if 'result_text' in locals() else 'N/A'}")
        return {"theses": [], "thesis_supports": [], "filename": filename}
    except Exception as e:
        print(f"Error analyzing {filename}: {e}")
        import traceback
        traceback.print_exc()
        return {"theses": [], "thesis_supports": [], "filename": filename}

def extract_theses_from_text_chunked(text: str, filename: str, chunk_size: int) -> Dict[str, Any]:
    """
    Process very long documents in chunks with overlap to avoid splitting theses.
    """
    chunks = []
    overlap = 10000  # Overlap between chunks to avoid splitting theses
    
    # Split into chunks with overlap
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        chunks.append(chunk)
        if i + chunk_size >= len(text):
            break
    
    print(f"    Processing {len(chunks)} chunks...")
    
    all_theses = []
    all_supports = []
    
    for i, chunk in enumerate(chunks):
        print(f"    Chunk {i+1}/{len(chunks)} ({len(chunk):,} chars)...")
        chunk_result = extract_theses_from_text(chunk, f"{filename}_chunk{i}")
        all_theses.extend(chunk_result.get("theses", []))
        all_supports.extend(chunk_result.get("thesis_supports", []))
    
    return {
        "theses": all_theses,
        "thesis_supports": all_supports,
        "filename": filename
    }

# Extract theses from all PDFs
all_extractions = {}
for filename, text in pdf_texts.items():
    print(f"\n{'='*60}")
    print(f"Extracting theses from: {filename}")
    print(f"{'='*60}")
    extraction = extract_theses_from_text(text, filename)
    all_extractions[filename] = extraction
    print(f"  ✓ Found {len(extraction.get('theses', []))} theses, {len(extraction.get('thesis_supports', []))} support relationships")

print(f"\n✓ Extraction complete for {len(all_extractions)} document(s)")



Extracting theses from: AI 20250923.pdf
  Analyzing document (45,833 characters)...
  ✓ Found 10 theses, 0 support relationships

Extracting theses from: Argonautic BioTech Presentation 20250412.pdf
  Analyzing document (28,462 characters)...
  ✓ Found 6 theses, 3 support relationships

Extracting theses from: Argonautic Consolidated Thesis Presentation 20250818.pdf
  Analyzing document (47,895 characters)...
  ✓ Found 9 theses, 5 support relationships

Extracting theses from: Argonautic Construction Technology 20250412.pdf
  Analyzing document (35,062 characters)...
  ✓ Found 13 theses, 3 support relationships

✓ Extraction complete for 4 document(s)


In [15]:
# Step 3: Deduplicate and number theses

def deduplicate_theses(all_extractions: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    Combine theses from all documents, deduplicate similar ones, and assign unique numbers.
    """
    all_theses = []
    thesis_to_number = {}
    thesis_number = 1
    
    # First pass: collect all theses with their source files
    for filename, extraction in all_extractions.items():
        for thesis_data in extraction.get("theses", []):
            thesis = thesis_data.get("thesis", "").strip()
            description = thesis_data.get("description", "").strip()
            
            if thesis:  # Only add non-empty theses
                all_theses.append({
                    "thesis": thesis,
                    "description": description,
                    "source_file": filename
                })
    
    # Second pass: deduplicate similar theses
    # Group theses by similarity (simple approach: exact match or very similar)
    unique_theses = []
    seen_theses = set()
    
    for thesis_data in all_theses:
        thesis = thesis_data["thesis"]
        thesis_lower = thesis.lower().strip()
        
        # Check if we've seen a very similar thesis
        is_duplicate = False
        for seen in seen_theses:
            # Simple similarity check: if one is contained in the other or very similar
            if (thesis_lower in seen.lower() or seen.lower() in thesis_lower) and abs(len(thesis_lower) - len(seen)) < 20:
                is_duplicate = True
                break
        
        if not is_duplicate:
            seen_theses.add(thesis)
            unique_theses.append(thesis_data)
            thesis_to_number[thesis] = thesis_number
            thesis_number += 1
    
    # Build thesis supports relationships with numbers
    thesis_supports = []
    for filename, extraction in all_extractions.items():
        for rel in extraction.get("thesis_supports", []):
            source = rel.get("source_thesis", "").strip()
            target = rel.get("target_thesis", "").strip()
            
            # Find the unique thesis numbers
            source_num = None
            target_num = None
            
            for thesis, num in thesis_to_number.items():
                if source and (source in thesis or thesis in source):
                    source_num = num
                if target and (target in thesis or thesis in target):
                    target_num = num
            
            if source_num and target_num:
                thesis_supports.append({
                    "source_thesis_number": source_num,
                    "target_thesis_number": target_num,
                    "source_thesis": source,
                    "target_thesis": target
                })
    
    return {
        "theses": unique_theses,
        "thesis_to_number": thesis_to_number,
        "thesis_supports": thesis_supports
    }

# Deduplicate and number
print("Deduplicating theses...")
deduplicated = deduplicate_theses(all_extractions)
print(f"✓ Found {len(deduplicated['theses'])} unique theses (from {sum(len(e.get('theses', [])) for e in all_extractions.values())} total)")
print(f"✓ Found {len(deduplicated['thesis_supports'])} support relationships")


Deduplicating theses...
✓ Found 32 unique theses (from 38 total)
✓ Found 11 support relationships


In [16]:
# Step 4: Create review-friendly CSV

def create_review_csv(deduplicated: Dict[str, Any], output_file: Path):
    """
    Create a CSV file optimized for human review.
    """
    thesis_to_number = deduplicated["thesis_to_number"]
    thesis_supports = deduplicated["thesis_supports"]
    
    # Build a map of which theses each thesis supports
    supports_map = {}
    for rel in thesis_supports:
        source_num = rel["source_thesis_number"]
        target_num = rel["target_thesis_number"]
        
        if source_num not in supports_map:
            supports_map[source_num] = []
        supports_map[source_num].append(target_num)
    
    # Create rows for CSV
    rows = []
    for thesis_data in deduplicated["theses"]:
        thesis = thesis_data["thesis"]
        thesis_num = thesis_to_number[thesis]
        
        # Get list of thesis numbers this thesis supports
        supports_list = sorted(supports_map.get(thesis_num, []))
        supports_str = ", ".join(map(str, supports_list)) if supports_list else ""
        
        rows.append({
            "Thesis Number": thesis_num,
            "Thesis Statement": thesis,
            "Description": thesis_data["description"],
            "Supports Thesis Numbers": supports_str,
            "Source File": thesis_data["source_file"]
        })
    
    # Sort by thesis number
    rows.sort(key=lambda x: x["Thesis Number"])
    
    # Create DataFrame and save
    df = pd.DataFrame(rows)
    df.to_csv(output_file, index=False)
    
    return df

# Create the review CSV
output_file = OUTPUT_FOLDER / "theses_for_review.csv"
df = create_review_csv(deduplicated, output_file)

print(f"\n✓ Created review CSV: {output_file}")
print(f"\nCSV Summary:")
print(f"  Total theses: {len(df)}")
print(f"  Theses with support relationships: {len(df[df['Supports Thesis Numbers'] != ''])}")
print(f"\nFirst few rows:")
print(df.head(10).to_string())
print(f"\n📝 Next steps:")
print(f"  1. Open {output_file} in Excel or a text editor")
print(f"  2. Review each thesis statement and description")
print(f"  3. Ensure descriptions are 2-3 sentences and accurate")
print(f"  4. Verify and update 'Supports Thesis Numbers' column")
print(f"  5. Save as 'theses_reviewed.csv' when done")



✓ Created review CSV: ..\data\analysis\theses_for_review.csv

CSV Summary:
  Total theses: 32
  Theses with support relationships: 9

First few rows:
   Thesis Number                                                                                                                                                            Thesis Statement                                                                                                                                                                                                                                                                                                                       Description Supports Thesis Numbers      Source File
0              1                     Generative AI's true impact is still in its early stages, with second order applications and AI-native business models continuing to emerge and evolve.                                                                            This thesis posits that the full 